In [1]:
import cv2
import itertools as it
import numpy as np
import pickle

def show_and_wait(frame, title='tesst', wait_time=0):
    cv2.imshow(title, frame)
    key = cv2.waitKey(wait_time)
    return None if key < 0 else chr(key)

def read_and_wait(video, title='tesst', wait_time=0):
    result, frame = video.read()
    if result:
        return show_and_wait(frame, title, wait_time)

In [2]:
# Load all hero's names.
file_path = r"C:\Users\cidzerda\Documents\GitHub\ctc\hero_names.txt"
with open(file_path) as fin:
    hero_names = [s.strip() for s in fin]
print(len(hero_names), sorted(set(it.chain.from_iterable(hero_names))), len(set(it.chain.from_iterable(hero_names))))

119 [' ', "'", '-', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'] 29


In [3]:
# Determine the smallest set of hero names that encompasses the alphabet.
def get_letters(hero_names):
    g = it.chain.from_iterable(hero_names)
    s = set(g)
    return s
def get_letter(letters, available_hero_names):
    def count_names(c):
        l = [s for s in available_hero_names if c in s]
        return len(l)
    d = {count_names(c): c for c in letters}
    n = min(d)
    return d[n]
def get_names(letter, available_hero_names):
    return [s for s in available_hero_names if letter in s]
def get_name(l, letters):
    def count_letters(s):
        s = letters - set(s)
        return len(s)
    d = {count_letters(s): s for s in l}
    n = min(d)
    return d[n]
def fn(hero_names):
    # Create the alphabet used in the hero names.
    all_letters = get_letters(hero_names)
    # Create a list of selected hero names, starting with the shortest and longest names.
    selected_hero_names = ['IO', 'KEEPER OF THE LIGHT']
    # Include the heros already gathered.
    selected_hero_names += ['BROODMOTHER', 'CLINKZ', 'JUGGERNAUT', 'NYX ASSASSIN', 'OUTWORLD DEVOURER', 'QUEEN OF PAIN']
    # Create a set of available hero names.
    available_hero_names = set(hero_names) - set(selected_hero_names)
    # Create a set of accounted letters.
    accounted_letters = set(it.chain.from_iterable(selected_hero_names))
    nletters = len(all_letters)
    while len(accounted_letters) < nletters:
        # Create a list of letters that are not represented in the list of selected names.
        letters = all_letters - accounted_letters
        # From that list, select the letter that is in the fewest names of those in the available names.
        letter = get_letter(letters, available_hero_names)
        # Get the list of names that contain that letter and are in the available names.
        names = get_names(letter, available_hero_names)
        # From that list, select the name with the greatest number of letters that are
        # not represented in the list of selected names.
        name = get_name(names, letters)
        # Add that name to the list of selected names.
        selected_hero_names.append(name)
        available_hero_names.remove(name)
        # Add that name's letters to the set of accounted letters.
        accounted_letters.update(name)
    return selected_hero_names
print(sorted(fn(hero_names)))

['ANTI-MAGE', 'BROODMOTHER', 'CLINKZ', 'IO', 'JUGGERNAUT', 'KEEPER OF THE LIGHT', "NATURE'S PROPHET", 'NYX ASSASSIN', 'OUTWORLD DEVOURER', 'QUEEN OF PAIN']


In [4]:
with open(r"C:\Users\cidzerda\Documents\GitHub\strevr\data\selected.txt") as fin:
    labels =  fin.readline()
with open(r"C:\Users\cidzerda\Documents\GitHub\strevr\data\selected.pickle", 'rb') as fin:
    images = [pickle.load(fin) for _ in labels]

# Move invalid images to a separate array.
invalid_images = images[-58:-54] + images[-45:-41] + images[-11:] # There are nineteen of these.
del images[-58:-54]
del images[-45:-41]
del images[-11:]

# Map all images to L*u*v*.
luvs = [cv2.cvtColor(image, cv2.COLOR_BGR2Luv) for image in images]

# Combine the list of three-dimensional tensors into a single four-dimensional tensor.
images = np.stack(images, axis=0)
luvs = np.stack(luvs, axis=0)

# There are 2981 valid images.  That number's prime factors are 11 and 271.
print(images.shape)

(2981, 6, 52, 3)


In [5]:
def fn(desired_ells, desired_factor, color):
    background = np.ones(desired_factor.shape, dtype=np.float32)
    background = np.stack([c * background for c in color], axis=-1)
    luv = cv2.cvtColor(background, cv2.COLOR_BGR2Luv)
    luv[:,:,0] = luv[:,:,0] * (1.0 - desired_factor) + desired_ells * desired_factor
    image = cv2.cvtColor(luv, cv2.COLOR_Luv2BGR)
    return image

# Extract all L*.
ells = luvs[:,:,:,0]

# Determine the desired L* by computing the mean of all L*.
mean = np.mean(ells, axis=0)
desired_ells = mean

# Determine the desired factor by scaling the range of standard deviations of all L* from [min, max] to [1, 0].
std = np.std(ells, axis=0)
desired_factor = (std - np.max(std)) / (np.min(std) - np.max(std))

# Display examples side-by-side.
random_indices = sorted(np.random.uniform(0, len(images), [9]).astype(int))
random_indices = [99, 558, 636, 1201, 2012, 2543, 2694, 2781, 2917] # These are good.
g = ((fn(desired_ells, desired_factor, images[i][3, 17]), images[i]) for i in random_indices)
l = list(it.chain.from_iterable(g))
show_and_wait(cv2.resize(np.vstack(l), (520, 60 * len(l))))

cv2.destroyAllWindows()